rebinning of the vertical velocity data on a new sst grid


In [1]:
# importing necessary libraries
import matplotlib.pyplot as plt
from matplotlib import rcParams
import matplotlib
import numpy as np
import xarray as xr
from datetime import datetime
import matplotlib.dates as mdates
import glob
#from myFunctions import lcl
#from myFunctions import f_closest
from warnings import warn
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
from scipy import interpolate
import custom_color_palette as ccp
from matplotlib import rcParams
import matplotlib.ticker as ticker


def f_interpolate_SST_and_merge(SST_DS, dataset_obs):
    '''function to interpolate SST values on the time resolution of the observations given as input
    input: 
    SST_DS: xarray dataset containing sst values
    dataset_obs: xarray dataset containing the observations to merge with sst data
    output: 
    data_merged: data returned 
    '''
    
    # interpolating sst data at 1 min resolution to the 10 s res of the wind lidar
    sst_data_interp = SST_DS.interp(time=dataset_obs['time'].values)

    # merging the interpolated dataset and the wind lidar dataset
    data_merged = xr.merge([dataset_obs, sst_data_interp])
    return(data_merged)


def f_calculate_binned_data(data_input, SST_binned_arr):
    
    ''' function to calculate mean values of all variables for each SST bin, for all instruments
    author: Claudia Acquistapace
    date: 20 Sept 2021
    input: - data_input: input xarray dataset containing the variables as a function of time, height, to be averaged
            - SST_binned_arr: numpy array of sst binned values for calculating the mean 
    output: dataset_concat: xarray dataset of concatenated values with mean profiles corresponding to the sst bins. A variable n_el counts the number of profiles averaged together
    '''
    # calculating mean quantities f
    dataset_mean = []

    data_input = data_input.load()

    # selecting all columns in the bin interval
    for ind_bin in range(len(SST_binned_arr)-1):

        # selecting slices of datasets columns with SST values in the selected bin
        DS_sliced = data_merged.where((data_input.SST > SST_binned_arr[ind_bin]) & (data_input.SST < SST_binned_arr[ind_bin+1]), drop=True)

        # add variable of the number of elements of the slice
        n_el = len(DS_sliced.SST.values)
        DS_sliced['n_elements'] = n_el

        # calculate mean profile averaging all selected time stamps together
        dataset_mean.append(DS_sliced.mean(dim='time', skipna=True))


    # concatenating datasets corresponding to SST bins on a new bin dimension
    dataset_concat = xr.concat([dataset_mean[i] for i in np.arange(len(dataset_mean))], dim='SST_binned')
    return(dataset_concat)




dict_plot_settings = {
    'labelsizeaxes':14,
    'fontSizeTitle':16,
    'fontSizeX'    :16,
    'fontSizeY'    :16,
    'cbarAspect'   :10,
    'fontSizeCbar' :16,
    'rcparams_font':['Tahoma'],
    'savefig_dpi'  :100,
    'font_size'    :22, 
    'grid'         :True}




data_path = '/Volumes/Extreme SSD/work/006_projects/001_Prec_Trade_Cycle/post_processed_data/diurnal_cycle_removed/'
tsg_file = "/Volumes/Extreme SSD/work/006_projects/001_Prec_Trade_Cycle/tsg_sst_data/tsg/nc/msm_089_1_tsg.nc"
path_out_plots = '/Volumes/Extreme SSD/work/006_projects/001_Prec_Trade_Cycle/SST_impact_work/plots_paper/'


In [2]:
# reading tsg file ( data with 1 min resolution)
tsg_data = xr.open_dataset(tsg_file)

# reading data containing flags to filter out rainy columns
flag_file_list = np.sort(glob.glob('/Volumes/Extreme SSD/work/006_projects/001_Prec_Trade_Cycle/post_processed_data/*_flags_cloud_properties.nc'))
flag_file_list = flag_file_list[13:15]

flag_data = xr.open_mfdataset(flag_file_list)

# reading tsg file ( data with 1 min resolution)
tsg_data = xr.open_dataset(tsg_file)

# identifying time stamps of sst corresponding to time stamps of radiosondes
t_start = datetime(2020, 2, 2, 0, 0, 0)
t_end = datetime(2020, 2, 3, 23, 59, 59)

# slicing tsg datase t for the selected time interval and extracting sst
sliced_tsg_ds = tsg_data.sel(TIME=slice(t_start, t_end))
tsg_sst = sliced_tsg_ds['TEMP'].values
tsg_time_sst = sliced_tsg_ds['TIME'].values
tsg_flag = sliced_tsg_ds['TEMP_QC'].values

# averaging together the sst of the different sst sensors for tsg
temp0 = sliced_tsg_ds.TEMP[:,0].values
temp1 = sliced_tsg_ds.TEMP[:,1].values
sst_tsg = temp0
sst_tsg[np.isnan(temp0)] = temp1[np.isnan(temp0)]

# producing output dataset of sst_tsg for the selected time window
# creating dataset with coordinates sst and height
dim_sst           = ['time']
coords         = {"time":sliced_tsg_ds.TIME.values}
SST              = xr.DataArray(dims=dim_sst, coords=coords, data=sst_tsg,
                 attrs={'long_name':'sea surface temperature ',
                        'units':'$^{\circ}$C'})
variables         = {'SST':SST}
SST_DS      = xr.Dataset(data_vars = variables,
                       coords = coords)


In [5]:
# building SST binned array
SST_min = np.nanmin(sst_tsg)
SST_max = np.nanmax(sst_tsg)
SST_binned_arr = np.arange(SST_min, SST_max, step=0.025)



# define new sst array for reducing the resolution of the sampling
def f_calc_new_sst(sst_arr):
    
    print(sst_arr)
    new_sst_arr = np.zeros(int(len(sst_arr)/2)+1)
    ind_new = 0
    for ind_sst in range(len(sst_arr)):
        if (ind_sst % 2) == 0:
            print(ind_sst)
            print(sst_arr[ind_sst])
            new_sst_arr[ind_new] = sst_arr[ind_sst]
            ind_new = ind_new+1
    print(new_sst_arr)
    return(new_sst_arr)
new_sst_arr =  f_calc_new_sst(SST_binned_arr)


# calculate label marks for bins
sst_bin_label = []
for ind in range(len(new_sst_arr)-1):
    sst_bin_label.append(round((new_sst_arr[ind]+new_sst_arr[ind+1])/2,2))
    


[26.283 26.308 26.333 26.358 26.383 26.408 26.433 26.458 26.483 26.508
 26.533 26.558 26.583 26.608 26.633 26.658 26.683 26.708 26.733 26.758
 26.783 26.808 26.833 26.858 26.883 26.908 26.933 26.958 26.983 27.008
 27.033 27.058 27.083 27.108 27.133 27.158 27.183 27.208 27.233 27.258
 27.283 27.308 27.333 27.358 27.383 27.408 27.433 27.458 27.483 27.508
 27.533 27.558 27.583 27.608 27.633 27.658]
0
26.283
2
26.333
4
26.382999999999996
6
26.432999999999993
8
26.48299999999999
10
26.532999999999987
12
26.582999999999984
14
26.63299999999998
16
26.68299999999998
18
26.732999999999976
20
26.782999999999973
22
26.83299999999997
24
26.882999999999967
26
26.932999999999964
28
26.98299999999996
30
27.03299999999996
32
27.082999999999956
34
27.132999999999953
36
27.18299999999995
38
27.232999999999947
40
27.282999999999944
42
27.33299999999994
44
27.38299999999994
46
27.432999999999936
48
27.482999999999933
50
27.53299999999993
52
27.582999999999927
54
27.632999999999925
[26.283 26.333 26.383 26

In [6]:
# reading all arthus data 

# variable list
var_list =['VW']

VW_dict = {
     'var_name'  : 'VW',
     'var_string': 'Vertical velocity',
     'var_units' : ' ms$^{-1}$',
     'var_min'   : -2.,
     'var_max'   : 2.,
     'thr_min'   : -5.,
     'thr_max'   : 5.,
     'avg_time'  : '15',
     'cmap'      : 'seismic',
     'title'     : 'Vertical velocity: 28/01-04/02'}


dict_list = [VW_dict]
#dict_list = [theta_dict, theta_e_dict]


vars_arthus = []

for i_var, dict_var in enumerate(dict_list):

    print('processing variable: ', dict_var['var_string'])
    print(data_path+dict_var['var_name']+'*.nc')
    
    # reading file list of the files for the selected variable
    arthus_file_list = np.sort(glob.glob(data_path+dict_var['var_name']+'_2020*.nc'))
    print(arthus_file_list)
    
    # read the two datasets together
    arthus_dataset = xr.open_mfdataset(arthus_file_list)
    
    # renaming variable time and height and dimension time and height (step necessary for next operations)
    #arthus_dataset = arthus_dataset.rename_dims({'Time':'time'})
    #arthus_dataset = arthus_dataset.rename_dims({'Height':'height'})
    #arthus_dataset = arthus_dataset.rename_vars({'Time':'time'})
    #arthus_dataset = arthus_dataset.rename_vars({'Height':'height'})

    print('files read')

    # interpolate flag on time resolution of arthus data, picking the closest time stamp to lidar time stamps
    flag_data_interp = flag_data.interp(time=arthus_dataset['time'].values, method='nearest')

    print('flag interpolated')
    
    # building a mask to filter out Ze rainy columns and substitute them with nans
    # set to nan the values out of the thresholds for the selected variable
    mask = np.zeros((len(arthus_dataset.time.values), len(arthus_dataset.height.values)))
    for ind in range(len(flag_data_interp.time.values)):
        if (flag_data_interp["flag_rain_ground"].values[ind] == 1) | (flag_data_interp["flag_rain"].values[ind] == 1):
            mask[ind,:] = np.repeat(1, len(arthus_dataset.height.values))

    print('mask calculated')
    
    arthus_dataset["nans"] = xr.full_like(arthus_dataset.product_no_diurnal_cycle, fill_value=np.nan)
    arthus_dataset['product_no_diurnal_cycle'] = xr.where(mask == 0, arthus_dataset['product_no_diurnal_cycle'], arthus_dataset["nans"])

    
    # interpolating SST data on the arthus data 
    arthus_all_SST = f_interpolate_SST_and_merge(SST_DS, arthus_dataset)
    print(arthus_all_SST)
    
    
    
    print('interpolation and merging done')

    # calculating mean quantities for each bin
    dataset_mean = []
    dataset_std = []
    dataset_n = []
    
    # selecting all columns in the bin interval
    for ind_bin in range(len(new_sst_arr)-1):

        # selecting slices of datasets columns with SST values in the selected bin
        DS_sliced = arthus_all_SST.where((arthus_all_SST.SST > new_sst_arr[ind_bin]) & (arthus_all_SST.SST < new_sst_arr[ind_bin+1]), drop=True)

        # add variable of the number of elements of the slice
        n_el = len(DS_sliced.SST.values)
        
        # add variable of absolute value of the var
        dims_ds = ['time','height']
        coords_ds = {'time':DS_sliced['time'].values, 'height':DS_sliced['height'].values}
        abs_val = np.abs(DS_sliced['product_no_diurnal_cycle'].values)
        DS_sliced['abs_val'] =  xr.DataArray(dims=dims_ds, coords=coords_ds, data=abs_val)

    
        # add variable of count of values for each sliced dataset where we calculate mean/std 
        dims_ds = ['time','height']
        coords_ds = {'time':DS_sliced['time'].values, 'height':DS_sliced['height'].values}
        counts = np.count_nonzero(~np.isnan(DS_sliced['product_no_diurnal_cycle'].values), axis=0)
        #print(np.shape(counts))
        #print(counts)
        DS_sliced['n_elements'] =  xr.DataArray(dims=['height'], \
                                                coords={'height':DS_sliced['height'].values}, \
                                                data=counts, 
                                               attrs={'long_name':'number of values in each bin SST/height'})        
        
        
        # calculate mean profile averaging all selected time stamps together
        dataset_mean.append(DS_sliced.mean(dim='time', skipna=True))
        dataset_std.append(DS_sliced.std(dim='time', skipna=True))
        dataset_n.append(DS_sliced['n_elements'])
        
        
    # concatenating datasets corresponding to SST bins on a new bin dimension
    arthus_SST_concat = xr.concat([dataset_mean[i] for i in np.arange(len(dataset_mean))], dim='SST_binned')
    arthus_SST_std_concat = xr.concat([dataset_std[i] for i in np.arange(len(dataset_std))], dim='SST_binned')
    arthus_SST_n_concat = xr.concat([dataset_n[i] for i in np.arange(len(dataset_n))], dim='SST_binned')
    
    
    # saving variable of interest in a dictionary with its name: for vertical velocity (VW) we save the absolute value
    # of vertical velocity for mean variable, and the std of the vertical velocity values.
    if dict_var['var_name']!= 'VW':
        dict_variable_nodc = {'var_name':dict_var['var_name'],
                     'var':arthus_SST_concat['product_no_diurnal_cycle'].values, 
                     'std':arthus_SST_std_concat['product_no_diurnal_cycle'].values,
                      'n':arthus_SST_n_concat.data}
    else:
        dict_variable_nodc = {'var_name':dict_var['var_name'],
                     'var':arthus_SST_concat['abs_val'].values, 
                     'std':arthus_SST_std_concat['product_no_diurnal_cycle'].values,
                      'n':arthus_SST_n_concat.data}       
    
    
    # append the dictionary in a list of dictionaries containing all arthus variables
    vars_arthus.append(dict_variable_nodc)


# converting the list of dictionaries in a dictionary called variables to be saved in ncdf
dims             = ['sst','height']
coords           = {'sst':new_sst_arr[0:-1], 'height':arthus_SST_concat['height'].values}
variables = {}
for i in range(len(dict_list)):
    key = vars_arthus[i]['var_name']
    value = xr.DataArray(dims=dims, coords=coords, data=vars_arthus[i]['var'],
                             attrs={'long_name':vars_arthus[i]['var_name']+' without diurnal cycle',
                                    'units':dict_var['var_units']})
    value_std = xr.DataArray(dims=dims, coords=coords, data=vars_arthus[i]['std'],
                             attrs={'long_name':'std of '+vars_arthus[i]['var_name']+' without diurnal cycle',
                                    'units':dict_var['var_units']})
    value_n = xr.DataArray(dims=dims, coords=coords, data=vars_arthus[i]['n'],
                             attrs={'long_name':'number of '+vars_arthus[i]['var_name']+' values in the bin',
                                    'units':'#'})
    variables[key] = value
    variables[key+'_std'] = value_std
    variables[key+'_n'] = value_n


global_attributes = {'CREATED_BY'       : 'Claudia Acquistapace',
                        'CREATED_ON'       :  str(datetime.now()),
                        'FILL_VALUE'       :  'NaN', 
                        'PI_NAME'          : 'Claudia Acquistapace',
                        'PI_AFFILIATION'   : 'University of Cologne (UNI), Germany', 
                        'PI_ADDRESS'       : 'Institute for geophysics and meteorology, Pohligstrasse 3, 50969 Koeln', 
                        'PI_MAIL'          : 'cacquist@meteo.uni-koeln.de',
                        'DATA_DESCRIPTION' : dict_var['var_string']+'with the diurnal cycle removed from the data',
                        'DATA_DISCIPLINE'  : 'Atmospheric Physics - Remote Sensing Lidar Profiler',
                        'DATA_GROUP'       : 'Experimental;Profile;Moving',
                        'DATA_SOURCE'      : 'arthus data',
                        'DATA_PROCESSING'  : 'https://github.com/ClauClouds/SST-impact/',
                        'INSTRUMENT_MODEL' : 'arthus raman lidar system',
                         'COMMENT'         : 'original data postprocessed by Diego Lange' }
dataset_out    = xr.Dataset(data_vars = variables,
                        coords = coords,
                        attrs = global_attributes)
dataset_out.to_netcdf('/Volumes/Extreme SSD/work/006_projects/001_Prec_Trade_Cycle/post_processed_data/binned_sst/arthus_binned_WV_coarser_sst.nc')


processing variable:  Vertical velocity
/Volumes/Extreme SSD/work/006_projects/001_Prec_Trade_Cycle/post_processed_data/diurnal_cycle_removed/VW*.nc
['/Volumes/Extreme SSD/work/006_projects/001_Prec_Trade_Cycle/post_processed_data/diurnal_cycle_removed/VW_20200202.nc'
 '/Volumes/Extreme SSD/work/006_projects/001_Prec_Trade_Cycle/post_processed_data/diurnal_cycle_removed/VW_20200203.nc']
files read
flag interpolated
mask calculated
<xarray.Dataset>
Dimensions:                   (height: 57, time: 16617)
Coordinates:
  * time                      (time) datetime64[ns] 2020-02-02T00:00:08 ... 2...
  * height                    (height) float32 225.0 275.0 ... 2975.0 3025.0
Data variables:
    product_no_diurnal_cycle  (time, height) float64 dask.array<chunksize=(8438, 57), meta=np.ndarray>
    product_no_noise          (time, height) float64 dask.array<chunksize=(8438, 57), meta=np.ndarray>
    nans                      (time, height) float64 dask.array<chunksize=(8438, 57), meta=np.ndarr

/Users/claudia/opt/anaconda3/lib/python3.8/site-packages/dask/array/numpy_compat.py:39: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
